In [1]:
import sys
if sys.version_info < (3, 6):
    raise ValueError('Você precisa de Python 3.6+ para continuar')

!pip3 install lark-parser


# Tutorial de Lark/JSON

Baseado em: https://github.com/lark-parser/lark/blob/master/docs/json_tutorial.md

Lark é um parser (analisador sintático), ou seja, um programa que aceita uma gramática e uma string de texto e produz uma árvore estruturada que representa esta string. Neste tutorial, escreveremos um analisador JSON no Lark e exploraremos os vários recursos do Lark no processo.

Este tutorial tem 5 partes.

1. Escrevendo a gramática
2. Criando o analisador
3. Moldando a árvore
4. Avaliando a árvore
5. Otimizando

Conhecimento assumido:

1. Python básico
2. Um entendimento básico de como usar expressões regulares


## Parte 1 - A Gramática

Lark aceita gramáticas em um formato chamado EBNF. Basicamente, é assim:

```
rule_name : lista de regras e TERMINAIS
          | outra lista possível de itens
          | etc.
TERMINAL  : "texto ou padrão comparado diretamente com o texto de entrada"
```
*(um terminal é uma string ou uma expressão regular)*

O analisador sintático tentará casar cada regra (parte esquerda), combinando seus itens (parte direita) sequencialmente, tentando cada alternativa e a partir daí construir uma expressão válida (na prática, o analisador é preditivo, portanto, não precisamos tentar todas as alternativas possíveis).

Como estruturar essas regras está além do escopo deste tutorial, mas geralmente é suficiente seguir a intuição. No caso do JSON, a estrutura é simples: um documento json é uma lista, um dicionário ou uma string, número, booleano etc. Os dicionários e listas são recursivos e contêm outros valores JSON que podem ser aninhados de forma arbitrária.

Vamos escrever essa estrutura no formato EBNF:

```
valor  : objeto
       | lista
       | STRING
       | NUMERO
       | "true" 
       | "false" 
       | "null"
 
lista  : "[" [valor ("," valor) *] "]"
objeto : "{" [par ("," par) *] "}"
par    : STRING ":" valor
```

Uma explicação rápida da sintaxe:

* Parênteses agrupam regras
* ``regra*`` significa zero ou mais repetições de uma regra.
* ``[regra]`` ou ``regra?`` significa que a regra é opcional (ou seja, pode aparecer ou não no texto).
* ``regra+`` significa uma ou mais repetições da regra.
* ``` `

Obviamente, ainda não definimos "STRING" e "NUMBER". Felizmente, esses dois literais já estão definidos na biblioteca comum de Lark e não precisamos escrever as regras explicitamente na gramática do Lark. https://json.org/ mostra a gramática do JSON e podemos ver que as regras para números e strings são justamente as mais complicadas da especificação.

```
%import common.ESCAPED_STRING -> STRING
%import common.SIGNED_NUMBER -> NUMERO
```

A seta (->) renomeia os terminais e usamos isto para utilizar nomes mas adequados que os escolhidos na biblioteca padrão. Também precisamos definir espaço em branco, que faz parte do texto. JSON ignora espaços em branco entre símbolos e podemos dizer para o Lark para ignorá-los quando aparecerem entre símbolos terminais ou não-terminais da gramática.

```
%import common.WS
%ignore WS
```

A propósito, se você está curioso para saber o que esses terminais significam, eles são aproximadamente equivalentes às expressões regulares

```
NUMBER : /-?\d+(\.\d+)?([eE][+-]?\d+)?/
STRING : /".*?(?<!\\)"/
WS     : /[ \t\n\f\r]+/
```

Lark aceita isso, se você realmente quer complicar sua vida :)

As definições originais em common.lark não aderem estritamente ao json.org - mas nosso objetivo aqui é aceitar o json, não validá-lo. Na realidade elas aceitam todos JSON válidos, mas não rejeitam alguns documentos JSON inválidos.

Observe que os terminais são escritos em MAIÚSCULAS, enquanto as regras não-terminais são escritas em minúsculas. Falaremos mais sobre as diferenças entre regras e terminais mais tarde.

### EXERCÍCIO

Implemente a gramática JSON mostrada acima utilizando o Lark

In [2]:
from lark import Lark

json_parser = Lark(r"""
valor : "null" // Coloque o resto da gramática do JSON aqui!
""", start="valor")

In [3]:
# Exemplo
tree = json_parser.parse('null')
print(tree.pretty())

valor



In [4]:
# Mais testes ...
exemplo_1 = "[1, 2, 3, 4]"
exemplo_2 = '{"x": 1, "y": 42}'
exemplo_3 = '[{"x": 1, "y": 42}, {"x": 0, "y": 0}]

json = exemplo_1
tree = json_parser.parse(json)
print(tree.pretty())

SyntaxError: EOL while scanning string literal (<ipython-input-4-5439781ae1a2>, line 4)

## Parte 2 - Criando o analisador

Depois de criar nossa gramática, criar o analisador é muito simples. Na verdade, se vocë conseguiu resolver o
exercício anterior, você já sabe tudo que esta seção vai falar.

Simplesmente instanciamos Lark e dizemos para aceitar um "valor":

In [ ]:
from lark import Lark

json_parser = Lark(r"""
valor  : objeto
       | lista
       | STRING
       | NUMERO
       | "true" 
       | "false" 
       | "null"

lista  : "[" [valor ("," valor) *] "]"
objeto : "{" [par ("," par) *] "}"
par    : STRING ":" valor

%import common.ESCAPED_STRING -> STRING
%import common.SIGNED_NUMBER -> NUMERO
%import common.WS
%ignore WS
""", start='valor')

É simples assim! Vamos testá-lo:

In [ ]:
json = '{"key": ["item0", "item1", 3,14]}'
tree = json_parser.parse(json)
tree

Podemos mostrar uma versão "melhorada" da árvore como o método `.pretty()` das árvores.

In [ ]:
print(tree.pretty())

Conforme prometido, o Lark cria automaticamente uma árvore que representa o texto analisado. Mas algo está faltando nesta árvore. Onde estão as chaves, vírgulas e todos os outros literais de pontuação?

O Lark filtra automaticamente os literais da árvore com base nos seguintes critérios:

* Remove terminais sem nome que aparecem como strings literais (ex., o `:` na regra `STRING ":" valor`).
* Remove terminais cujo nome começa com um sublinhado.
* Mantêm expressões regulares, mesmo as sem nome, a menos que o nome comece com um sublinhado.

Infelizmente, isso significa que ele também filtrará literais como "true" e "false", e perderemos essa informação. A próxima seção, "Moldando a árvore", trata desse problema e de outros.


##  Parte 3 - Moldando a árvore

Agora temos um analisador sintático que cria uma árvore sintática abstrata (ou: AST, do inglës *abstract syntax tree*), mas a árvore tem alguns problemas:

* "true", "false" e "null" são filtrados (teste você mesmo!)
* Árvore tem ramos inúteis, como valor, que atrapalham nossa visão.

Vou apresentar a solução e depois explicá-la:

```
?valor : objeto
       | lista
       | STRING  -> string 
       | NUMERO  -> numero
       | "true"  -> true
       | "false" -> false
       | "null"  -> null

...
```

1. As setas `->` denotam apelidos, que são nomes alternativos para uma regra específica. Nesse caso, nomearemos "true" / "false" / "null" para não perdermos informação sobre estes valores. Também apelidamos STRING e NUMERO para processá-los mais tarde.

2. O ponto de interrogação antes de uma regra (`?valor`) diz ao construtor de árvores para substituir um nó de valor pelo nó filho se ele tiver apenas um membro. Na nossa gramática, "valor" sempre possui apenas um membro e sempre será incorporado.

### EXERCÍCIO

Recrie a gramática com as novas alterações mostradas acima. Teste a gramática com o seguinte documento,

```json
{"key": ["item0", "item1", 3.14, true]}
```

o resultado deve ser uma árvore com o formato abaixo

```
objeto
  par
    string	"key"
    lista
      string	"item0"
      string	"item1"
      numero	3.14
      true
```

In [6]:
json_parser = Lark(r"""
?valor : objeto
       | lista
       | STRING  -> string
       | NUMERO  -> numero
       | "true"  -> true
       | "false" -> false
       | "null"  -> null

lista  : "[" [valor ("," valor) *] "]"
objeto : "{" [par ("," par) *] "}"
par    : STRING ":" valor

%import common.ESCAPED_STRING -> STRING
%import common.SIGNED_NUMBER -> NUMERO
%import common.WS
%ignore WS
""", start='valor')

In [ ]:
# Crie sua gramática aqui!

json_parser = ...

tree = json_parser.parse('{"key": ["item0", "item1", 3.14, true]}')
print(tree.pretty())

## Parte 4 - Avaliando a árvore

É bom ter uma árvore sintática, mas o que realmente queremos é um objeto JSON. A maneira de fazer isso é transformar a árvore, usando a classse Transformer.

Um transformador é uma classe com métodos correspondentes aos nomes das regras não-terminais. Para cada ramificação, o método apropriado será chamado com os filhos da ramificação como argumento e seu valor de retorno substituirá a ramificação na árvore.

Vamos escrever um transformador parcial, que lida com os valores atômicos

In [7]:
from lark import InlineTransformer

class JSONTransformer(InlineTransformer):
    def string(self, st):
        return st[1:-1]  # remove as aspas
    
    def numero(self, n):
        return float(n)    
    
    def true(self): 
        return True
    
    def false(self): 
        return False
    
    def null(self): 
        return None

E quando executamos, obtemos o seguinte:

In [8]:
tree = json_parser.parse('{"key": ["item0", "item1", 3.14, true]}')

transformer = JSONTransformer()
new = transformer.transform(tree)
print(new.pretty())

objeto
  par
    "key"
    lista
      item0
      item1
      3.14
      True



Parece bom :)

Vamos escrever um transformador completo que também possa lida com valores compostos como listas e objetos. O `InlineTransformer` do Lark chama cada função correspondente a uma regra de produção não-terminal com o número de argumentos igual ao número de filhos. Assim, o exemplo anterior chamaria a regra de lista com 4 argumentos correspondendo a "item0", "item1", 3.14 e True. 

Em Python, podemos dizer que uma função recebe um número variável de argumentos usando a notação de "splice". 
Basta escrever o nome do último argumento prefixado com um asterisco e o Python construirá uma tupla com todos os argumentos adicionais passados para a função.

In [9]:
def soma(x, y, *args):
    # Args é uma tupla que guarda todos argumentos adicionais
    return x + y + sum(args)

# Teste alguns casos
soma(1, 2, 3, 4)

10

Vamos usar isto para criar nosso transformer (aproveitamos e convertemos algumas funções para lambdas, que são uma sintaxe alternativa para funções simples em Python).

In [10]:
from lark import InlineTransformer

class JSONTransformer(InlineTransformer):
    # Os lambdas fazem o mesmo que as funções definidas anteriormente
    string = lambda _, st: st[1:-1]  # remove as aspas
    numero = float
    true = lambda _: True
    false = lambda _: False
    null = lambda _: None

    def lista(self, *itens):
        return list(itens)
    
    def par(self, chave, valor):
        return (self.string(chave), valor)
    
    def objeto(self, *pares):
        return dict(pares)

E quando executamos...

In [11]:
tree = json_parser.parse('{"key": ["item0", "item1", 3.14, true]}')

transformer = JSONTransformer()
transformer.transform(tree)

{'key': ['item0', 'item1', 3.14, True]}

Ótimo ;-)

## Parte 5 - Otimizando

### Etapa 1 - Referência

Até agora, temos um analisador JSON totalmente funcional, que aceita uma string JSON e retorna sua representação Pythonica. Mas qual é a velocidade?

Existem, é claro,bibliotecas JSON para Python escritas em C e nunca podemos competir com elas. Isso é aplicável a qualquer analisador que você escreveria em Lark, mas vamos testar os limites onde o Lark pode chegar.

O primeiro passo para otimizar é ter uma referência. Para fazer o benchmark, vamos pegar os dados do https://www.json-generator.com/. Mudando `{{repeat(5, 7)}` para `{{repeat(1000)}`, gera um arquivo JSON de aproximadamente 1,0MB. Baixamos este arquivo e salvamos como `teste.json`.

Vamos medir o tempo de execução para processar este arquivo com o Lark e comparar com a biblioteca padrão do Python.

In [12]:
import json

data = open('teste.json').read()

In [13]:
# Testamos o json

%timeit -n1 -r1 json.loads(data)

17.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
# Agora o Lark
def loads(json):
    tree = json_parser.parse(json)
    return JSONTransformer().transform(tree)

%timeit -n1 -r1 loads(data)

23.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
grammar = r"""
valor  : objeto
       | lista
       | STRING
       | NUMERO
       | "true" 
       | "false" 
       | "null"

lista  : "[" [valor ("," valor) *] "]"
objeto : "{" [par ("," par) *] "}"
par    : STRING ":" valor

%import common.ESCAPED_STRING -> STRING
%import common.SIGNED_NUMBER -> NUMERO
%import common.WS
%ignore WS
"""

json_parser = Lark(grammar, start="valor", parser="lalr")

In [16]:
# Agora o Lark
def loads(json):
    tree = json_parser.parse(json)
    return JSONTransformer().transform(tree)

%timeit -n1 -r1 loads(data)

2.16 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
json_parser = Lark(grammar, start="valor", parser="lalr", transformer=JSONTransformer())

# Agora o Lark
def loads(json):
    return json_parser.parse(json)

%timeit -n1 -r1 loads(data)

1.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
